<a href="https://colab.research.google.com/github/Mani117/Panda_Basics/blob/master/NCAP_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import os
import time
from google.colab import drive, files

In [0]:
drive.mount("/mntDrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /mntDrive


In [0]:
asean_segments = ["Mini Car", "Small Family Car", "Medium Family Car", "Large Family Car", "SUV", "MPV", "Pickup", "Small Lorry", "Semi Panel Van"]
latin_ncap_weblink = "https://www.latinncap.com/en/results"
asean_ncap_baselink = "http://www.aseancap.org/v2/?s=xxxxxxxxxx&post_type=car_result"
asean_ncap_weblink = [asean_ncap_baselink.replace("xxxxxxxxxx", segment.replace(" ", "+")) for segment in asean_segments]
# temp_link = "https://www.latinncap.com/en/result/80/byd-f0-no-airbags"

In [0]:
user_agent_file = open("/mntDrive/My Drive/Colab Notebooks/user_agent", "r")
agent_list = np.array([agents for agents in user_agent_file])
np.random.shuffle(agent_list)
np.random.shuffle(agent_list)
np.random.shuffle(agent_list)
np.random.shuffle(agent_list)
np.random.shuffle(agent_list)
length = len(agent_list)
def user_agent_ret():
    global length
    global agent_list
    num = np.random.randint(0, length-1)
    return(agent_list[num].strip())

## LATIN NCAP

In [0]:
actual_page = requests.get("https://www.euroncap.com/en/results/skoda/superb/11060", headers = {'User-Agent': user_agent_ret()})
if(actual_page.status_code != 200):
    print("Latin NCAP No Result")
else:
    soup = BeautifulSoup(actual_page.text, "html.parser")

In [0]:
temp_data = soup.prettify()
temp_file = open("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP_2009.html", "w")
temp_file.write(temp_data)
temp_file.close()
files.download("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP_2009.html")

In [0]:
latin_ncap_tp_link = []
for dat_div in soup.find_all("div", {"class":"fila "}):
    latin_ncap_tp_link.append(dat_div.find_all("a", {"class":"btn-mas"})[0]["href"])
latin_ncap_tp_link = ["https://www.latinncap.com/"+links for links in latin_ncap_tp_link]

In [0]:
#latin_columns = ['vehicle_name', 'model_date', 'Body_type', 'Made_in','Publication_year', 'Crash_test_weight', 'adult_rating', 'child_rating', 'adult_score', 'child_score', 'Test_valid_for', 'Driver_frontal_airbag', 'Passenger_frontal_airbag', 'Side_head_airbags', 'Side_body_airbags_(front)', 'Driver_knee_airbag', 'Front_seatbelt_pretensioners', 'Passenger_Front_seatbelt_pretensioners', 'SBR', 'ISOFIX', 'ABS', 'ESC_(UN13_or_GTR8)', 'Bodyshell_integrity', 'Side_impact_protection_(Structural)']
latin_columns = ['vehicle_name', 'model_date', 'Body_type', 'Made_in','Publication_year', 'Crash_test_weight', 'adult_rating', 'child_rating', 'adult_score', 'child_score']

In [0]:
def latin_ncap_scrapper(html_text):
    adult_rating = 0
    child_rating = 0
    vehicle_details = {}
    vehicle_details_temp_list = []
    vehicle_details_temp_list_name = []
    temp_check = 0
    soup = BeautifulSoup(html_text, "html.parser")


    for data in soup.find_all("div", {"class":"adulto"}):
        for div in data.find_all("div"):
            if(div["class"][0] == "estrella-act"):
                adult_rating += 1
        for span in data.find_all("span"):
            adult_score = float(span.text.split()[0])

    for data in soup.find_all("div", {"class":"nino"}):
        for div in data.find_all("div"):
            if(div["class"][0] == "estrella-act"):
                child_rating += 1
        for span in data.find_all("span"):
            child_score = float(span.text.split()[0])


    vehicle_details["adult_rating"] = adult_rating
    vehicle_details["child_rating"] = child_rating
    vehicle_details["adult_score"] = adult_score
    vehicle_details["child_score"] = child_score


    for div in soup.find_all("div", {"class": "caja-info"}):
        for h4 in div.find_all("h4"):
            vehicle_name = h4.text.strip()
        for p in div.find_all("p"):
            date = p.text.strip()

    vehicle_details["vehicle_name"] = vehicle_name
    vehicle_details["model_date"] = date


    for div in soup.find_all("div", {"class": "caja-tabla"}):
        for div2 in div.find_all("p"):
            if(temp_check == 0):
                vehicle_details_temp_list_name.append(div2.text.strip())
                temp_check = 1
            else:
                vehicle_details_temp_list.append(div2.text.strip())
                temp_check = 0
    vehicle_details_list = vehicle_details_temp_list[0:18]
    vehicle_details_temp_list_name = vehicle_details_temp_list_name[0:18]
    vehicle_details_list_name = ["_".join(val.split()) for val in vehicle_details_temp_list_name]

    for head, val in zip(vehicle_details_list_name, vehicle_details_list):
        vehicle_details[head] = val

    try:
        vehicle_weight = int(re.split('[kgKG]{2}', vehicle_details["Crash_test_weight"].replace(".", ""), flags=re.IGNORECASE)[0])
    except:
        vehicle_weight = int(re.split('[kgKG]{2}', vehicle_details["Crash_test_weight"].replace(".", ""), flags=re.IGNORECASE)[1])
    
    vehicle_details["Crash_test_weight"] = vehicle_weight
        
    return(vehicle_details)

In [0]:
#df_latin = pd.DataFrame(columns = ['vehicle_name', 'model_date', 'Body_type', 'Made_in','Publication_year', 'Crash_test_weight', 'adult_rating', 'child_rating', 'adult_score', 'child_score', 'Test_valid_for', 'Driver_frontal_airbag', 'Passenger_frontal_airbag', 'Side_head_airbags', 'Side_body_airbags_(front)', 'Driver_knee_airbag', 'Front_seatbelt_pretensioners', 'Passenger_Front_seatbelt_pretensioners', 'SBR', 'ISOFIX', 'ABS', 'ESC_(UN13_or_GTR8)', 'Bodyshell_integrity', 'Side_impact_protection_(Structural)'])
df_latin = pd.DataFrame(columns = ['vehicle_name', 'model_date', 'Body_type', 'Made_in','Publication_year', 'Crash_test_weight', 'adult_rating', 'child_rating', 'adult_score', 'child_score'])
for latin_link in latin_ncap_tp_link:
    print(latin_link)
    raw_latin_page = requests.get(latin_link, headers = {'User-Agent': user_agent_ret()})
    if(raw_latin_page.status_code != 200):
        print("Latin NCAP No Result")
    else:
        current_latin_veh_data = latin_ncap_scrapper(raw_latin_page.text)
    # print(current_latin_veh_data)
    temp_list = []
    for cols in latin_columns:
        try:
            temp_list.append(current_latin_veh_data[cols])
        except KeyError:
            temp_list.append("Not_given")
    df_latin.loc[len(df_latin)] = temp_list
    time.sleep(4)

In [0]:
df_latin.to_csv("/mntDrive/My Drive/Colab Notebooks/Latin_NCAP.csv")
files.download("/mntDrive/My Drive/Colab Notebooks/Latin_NCAP.csv")

## ASEAN NCAP

In [0]:
df_asean = pd.DataFrame(columns = ['Vehicle_Name', 'About', 'Adult_Occupant', 'Child_Occupant', 'Safety_Assist', 'VARIANT', 'YEAR_BUILT', 'MODEL_YEAR', 'VEHICLE_CATEGORY', 'ENGINE_CAPACITY', 'KERB_MASS', 'TEST_LAB', 'OFFSET_FRONTAL_TEST', 'SIDE_IMPACT_TEST', 'HEAD_PROTECTION_TECHNOLOGY_EVALUATION*'])
asean_columns = ['Vehicle_Name', 'About',  'Adult_Occupant', 'Child_Occupant', 'Safety_Assist', 'VARIANT', 'YEAR_BUILT', 'MODEL_YEAR', 'VEHICLE_CATEGORY', 'ENGINE_CAPACITY', 'KERB_MASS', 'TEST_LAB', 'OFFSET_FRONTAL_TEST', 'SIDE_IMPACT_TEST', 'HEAD_PROTECTION_TECHNOLOGY_EVALUATION*']

In [0]:
def All_Rating(AOP, COP, Safety_Assist):
    max_AOP = 36
    max_COP = 49
    max_Safety_Assist = 18
    AOP_score = round((AOP/max_AOP)*100, 2)
    COP_score = round((COP/max_COP)*100, 2)
    Safety_Assist = round((Safety_Assist/max_Safety_Assist)*100, 2)
    Total_Score_Percent = (0.5*AOP_score) + (0.25*COP_score) + (0.25*Safety_Assist)
    
    if(Total_Score_Percent > 75):
        over_all = 5
    elif(Total_Score_Percent > 65):
        over_all = 4
    elif(Total_Score_Percent > 50):
        over_all = 3
    elif(Total_Score_Percent > 40):
        over_all = 2
    elif(Total_Score_Percent > 30):
        over_all = 1
    else:
        over_all = 0
        
    if(AOP_score > 75):
        AOP_rating = 5
    elif(AOP_score > 65):
        AOP_rating = 4
    elif(AOP_score > 45):
        AOP_rating = 3
    elif(AOP_score > 30):
        AOP_rating = 2
    elif(AOP_score > 20):
        AOP_rating = 1
    else:
        AOP_rating = 0
    
    if(COP_score > 75):
        COP_rating = 5
    elif(COP_score > 60):
        COP_rating = 4
    elif(COP_score > 30):
        COP_rating = 3
    elif(COP_score > 25):
        COP_rating = 2
    elif(COP_score > 15):
        COP_rating = 1
    else:
        COP_rating = 0
        
    if(Safety_Assist > 60):
        Safety_Assist_rating = 5
    elif(Safety_Assist > 40):
        Safety_Assist_rating = 4
    elif(Safety_Assist > 30):
        Safety_Assist_rating = 3
    elif(Safety_Assist > 20):
        Safety_Assist_rating = 2
    elif(Safety_Assist > 10):
        Safety_Assist_rating = 1
    else:
        Safety_Assist_rating = 0

    return([over_all, AOP_rating, COP_rating, Safety_Assist_rating])



def ASEAN_NCAP_Scrapper(html_text):
    Spec_id = []
    Spec_value = []
    Adult_occ_id = []
    Adult_occ_val = []
    Rating_Key = []
    Rating_Val = []
    temp_id = 0
    final_veh_dict = {}
    soup = BeautifulSoup(html_text, "html.parser")

    for h1 in soup.find_all("h1", {"class":"title"}):
        final_veh_dict["Vehicle_Name"] = h1.text.strip()

    for span in soup.find_all("span", {"class":"car-class"}):
        final_veh_dict["About"] = span.text.strip()

    for div in soup.find_all("div", {"class":"result_details"}):
        for div2 in div.find_all("div", {"class": "value"}):
            Rating_Val.append(div2.text.strip())
        for div3 in div.find_all("div", {"class": "rating-title"}):
            Rating_Key.append(div3.p.text.strip())

    #Rating_Val_converted = [float(x) for x in Rating_Val]
    Rating_Val_converted = Rating_Val

    for i, tab in enumerate(soup.find_all("table", {"class":"alignleft"})):
        if(i == 0):
            for td in tab.find_all("td"):
                if(temp_id == 0):
                    Spec_id.append(td.text.strip())
                    temp_id = 1
                else:
                    Spec_value.append(td.text.strip())
                    temp_id = 0
        if(i == 1):
            temp_id = 0
            for td in tab.find_all("td"):
                if(temp_id == 0):
                    Adult_occ_id.append(td.text.strip())
                    temp_id = 1
                else:
                    Adult_occ_val.append(td.text.strip())
                    temp_id = 0

    all_keys = Rating_Key + Spec_id + Adult_occ_id
    all_vals = Rating_Val_converted + Spec_value + Adult_occ_val
    all_keys_modified = ["_".join(key.split()) for key in all_keys]
#     Vehicle_rating = All_Rating(Rating_Val_converted[0], Rating_Val_converted[1], Rating_Val_converted[2])
#     final_veh_dict["Overall_Rating"] = Vehicle_rating[0]
#     final_veh_dict["AOP_Rating"] = Vehicle_rating[1]
#     final_veh_dict["COP_Rating"] = Vehicle_rating[2]
#     final_veh_dict["Safety_Assist_Rating"] = Vehicle_rating[3]

    for key, val in zip(all_keys_modified, all_vals):
        final_veh_dict[key] = val

    try:
        vehicle_weight = int(re.split('[kgKG]{2}', final_veh_dict["KERB_MASS"].replace(".", ""), flags=re.IGNORECASE)[0])
    except:
        vehicle_weight = int(re.split('[kgKG]{2}', final_veh_dict["KERB_MASS"].replace(".", ""), flags=re.IGNORECASE)[1])
    final_veh_dict["KERB_MASS"] = vehicle_weight
    return(final_veh_dict)

In [0]:
all_asean_ncap_link = []

In [0]:
for type_page in asean_ncap_weblink:
    temp_asean_list = []
    temp_asean_page = requests.get(type_page, headers = {'User-Agent': user_agent_ret()})
    soup = BeautifulSoup(temp_asean_page.text, "html.parser")
    for div in soup.find_all("div", {"class": "rating-table-row-c"}):
        try:
            temp_asean_list.append(div.a["href"])
        except:
            pass
    all_asean_ncap_link.extend(list(set(temp_asean_list)))
    time.sleep(3)

KeyboardInterrupt: ignored

In [0]:
for vehilce_page_link in all_asean_ncap_link:
    print(vehilce_page_link)
    each_veh_list = []
    veh_asean_data = requests.get(vehilce_page_link, headers = {'User-Agent': user_agent_ret()})
    asean_vehicle_data_dict = ASEAN_NCAP_Scrapper(veh_asean_data.text)
    for column in asean_columns:
        if(asean_vehicle_data_dict):
            try:
                each_veh_list.append(asean_vehicle_data_dict[column])
            except KeyError:
                each_veh_list.append("Not Present")
    if(each_veh_list):
        df_asean.loc[len(df_asean)] = each_veh_list
    time.sleep(4)

In [0]:
df_asean.to_csv("/mntDrive/My Drive/Colab Notebooks/ASEAN_NCAP.csv")
files.download("/mntDrive/My Drive/Colab Notebooks/ASEAN_NCAP.csv")

## EURO NCAP

In [0]:
# from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# import time
# from bs4 import BeautifulSoup


# binary = r'C:\Program Files\Mozilla Firefox\firefox.exe'
# options = Options()
# #options.set_headless(headless=True)
# options.binary = binary
# cap = DesiredCapabilities().FIREFOX
# cap["marionette"] = True #optional
# driver = webdriver.Firefox(options=options, capabilities=cap, executable_path=r"C:/Users/253693/Downloads/geckodriver-v0.26.0-win64/geckodriver.exe")
# driver_2 = webdriver.Firefox(options=options, capabilities=cap, executable_path=r"C:/Users/253693/Downloads/geckodriver-v0.26.0-win64/geckodriver.exe")
# driver.implicitly_wait(100)


# driver.get("https://www.euroncap.com/en/ratings-rewards/latest-safety-ratings")
# #elem = driver.find_element_by_xpath("//p[@class='filter-title']/label/input").click()
# elem = driver.find_element_by_class_name("checkbox").click()
# time.sleep(20)

# # sp = open("C:/Users/253693/Desktop/abc.html", "w", encoding = "utf8")
# # sp.write(driver.page_source)
# # sp.close()
# print ("Headless Firefox Initialized")


# add_all_link = []

# soup = BeautifulSoup(driver.page_source, "html.parser")
# for all_options in soup.find_all("div", {"class":"rating-table-row-c"}):
#     for cars in all_options.find_all("a"):
#         add_all_link.append(cars["href"])
        
# for all_options in soup.find_all("a", {"class":"rating-table-row-link"}):
#     add_all_link.append(all_options["href"])
        
# print("Done")

# # outside_div = driver.find_element_by_class_name("rating-table-body")
# # # all_options = element.find_elements_by_tag_name("option")
# driver.quit()

# base_euro_link = "https://www.euroncap.com"
# temp_link_list = []
# for link_str in add_all_link:
#     temp_link_list.append(base_euro_link + link_str)
    
# with open("C:/Users/253693/Desktop/Euro_ENCAP_ALL.txt", "w", encoding = "utf8") as some_file:
#     for links in temp_link_list:
#         some_file.write(links)
#         some_file.write("\n")

In [0]:
all_link_text = files.upload()

Saving Euro_ENCAP_ALL.txt to Euro_ENCAP_ALL.txt


In [0]:
euro_all_link =[]
for lines in str(all_link_text["Euro_ENCAP_ALL.txt"])[2:-1].split("\\r\\n"):
    euro_all_link.append(lines)
euro_all_link

['https://www.euroncap.com/en/results/aiways/u5/39672',
 'https://www.euroncap.com/en/results/audi/q8/39673',
 'https://www.euroncap.com/en/results/ford/puma/39674',
 'https://www.euroncap.com/en/results/mg/hs/39677',
 'https://www.euroncap.com/en/results/mg/zs-ev/39678',
 'https://www.euroncap.com/en/results/nissan/juke/39679',
 'https://www.euroncap.com/en/results/seat/mii/39972',
 'https://www.euroncap.com/en/results/vw/golf/39844',
 'https://www.euroncap.com/en/results/vw/up!/39809',
 'https://www.euroncap.com/en/results/skoda/citigo/39973',
 'https://www.euroncap.com/en/results/audi/q7/38855',
 'https://www.euroncap.com/en/results/ford/kuga/39116',
 'https://www.euroncap.com/en/results/ford/mondeo/39121',
 'https://www.euroncap.com/en/results/jeep/renegade/39123',
 'https://www.euroncap.com/en/results/peugeot/2008/39125',
 'https://www.euroncap.com/en/results/peugeot/2008/39126',
 'https://www.euroncap.com/en/results/porsche/taycan/39129',
 'https://www.euroncap.com/en/results/ren

In [0]:
euro_list_new = []
euro_list_old = []
i = 0
for link in euro_all_link:
    if("15748" in link):
        i = 1
    if(i == 0):
        euro_list_new.append(link)
    else:
        euro_list_old.append(link)

euro_list_old = euro_list_old[:-1]
euro_list_new

['https://www.euroncap.com/en/results/aiways/u5/39672',
 'https://www.euroncap.com/en/results/audi/q8/39673',
 'https://www.euroncap.com/en/results/ford/puma/39674',
 'https://www.euroncap.com/en/results/mg/hs/39677',
 'https://www.euroncap.com/en/results/mg/zs-ev/39678',
 'https://www.euroncap.com/en/results/nissan/juke/39679',
 'https://www.euroncap.com/en/results/seat/mii/39972',
 'https://www.euroncap.com/en/results/vw/golf/39844',
 'https://www.euroncap.com/en/results/vw/up!/39809',
 'https://www.euroncap.com/en/results/skoda/citigo/39973',
 'https://www.euroncap.com/en/results/audi/q7/38855',
 'https://www.euroncap.com/en/results/ford/kuga/39116',
 'https://www.euroncap.com/en/results/ford/mondeo/39121',
 'https://www.euroncap.com/en/results/jeep/renegade/39123',
 'https://www.euroncap.com/en/results/peugeot/2008/39125',
 'https://www.euroncap.com/en/results/peugeot/2008/39126',
 'https://www.euroncap.com/en/results/porsche/taycan/39129',
 'https://www.euroncap.com/en/results/ren

In [0]:
print(euro_list_old)
print(euro_list_new)

['https://www.euroncap.com/en/results/alfa-romeo/mito/15748', 'https://www.euroncap.com/en/results/dacia/sandero/15753', 'https://www.euroncap.com/en/results/dacia/sandero/18373', 'https://www.euroncap.com/en/results/ford/fiesta/15756', 'https://www.euroncap.com/en/results/ford/ka/15757', 'https://www.euroncap.com/en/results/ford/ranger/15759', 'https://www.euroncap.com/en/results/honda/accord/15760', 'https://www.euroncap.com/en/results/mercedes-benz/viano/15766', 'https://www.euroncap.com/en/results/opel/vauxhall/insignia/15770', 'https://www.euroncap.com/en/results/peugeot/308cc/15771', 'https://www.euroncap.com/en/results/renault/megane/15774', 'https://www.euroncap.com/en/results/volvo/xc60/15778', 'https://www.euroncap.com/en/results/vw/golf/15779', 'https://www.euroncap.com/en/results/daihatsu/cuore/15754', 'https://www.euroncap.com/en/results/hyundai/i30/15762', 'https://www.euroncap.com/en/results/lancia/delta/15764', 'https://www.euroncap.com/en/results/mercedes-benz/m-class/

In [0]:
def euro_ncap_scrapper(raw_file):
    data_dict = {}
    rating_title = []
    rating_score = []
    details_title = []
    details_score = []
    All_Data = {}
    Other_dict = {}
    CVS_dict = {}

#     raw_file = open("C:/Users/253693/Desktop/euro_ncap_raw_1.html", "r", encoding = "utf8")
    soup_tmp = BeautifulSoup(raw_file, "html.parser")
    soup = BeautifulSoup(soup_tmp.prettify(), "html.parser")

    for stars in soup.find_all("div", {"class": "stars"}):
        for img in stars.find_all("img"):
            com_link = img["data-src"]
            data_dict["star_rating"] = com_link[com_link.index("stars")+5]

    for rating in soup.find_all("div", {"class": "details"}):
        for p in rating.find_all("p"):
            rating_title.append(p.text.strip())
        for div in rating.find_all("div", {"class": "value"}):
            rating_score.append(div.text.strip())
    rating_title = rating_title[:4]

    for table1 in soup.find_all("span", {"class": "tcol1"}):
        details_title.append(table1.text.strip())
    for table2 in soup.find_all("span", {"class": "tcol2"}):
        details_score.append(table2.text.strip())

    for a, b  in zip(rating_title, rating_score):
        data_dict[a] = b
    for a, b  in zip(details_title, details_score):
        data_dict[a] = b

    try:
        vehicle_weight = int(re.split('[kgKG]{2}', data_dict["Kerb Weight"].replace(".", ""), flags=re.IGNORECASE)[0])
    except:
        vehicle_weight = int(re.split('[kgKG]{2}', data_dict["Kerb Weight"].replace(".", ""), flags=re.IGNORECASE)[1])
    data_dict["Kerb Weight"] = vehicle_weight


    for headers in soup.find_all("div", {"class": "accordion-item-header"}):
        key = re.split("([\n]+)", headers.text.strip())[0].strip()
        val = re.split("([\n]+)", headers.text.strip())[4].strip()
        CVS_dict[key] = val
    for headers in soup.find_all("div", {"class": "frame-header"}):
        key = re.split("([\n]+)", headers.text.strip())[0].strip()
        val = re.split("([\n]+)", headers.text.strip())[4].strip()
        Other_dict[key] = val
    All_Data.update(CVS_dict)
    All_Data.update(Other_dict)
    All_Data
    for key in All_Data:
        if("/" in  All_Data[key]):
            All_Data[key] = All_Data[key].split("/")[1].strip().split(" ")[0]
        else:
            All_Data[key] = All_Data[key].split(" ")[0]
    All_Data_key = {}
    for key in All_Data:
        key_mod = " ".join(key.split(" ")[:3])
        All_Data_key[key_mod] = All_Data[key]
    data_dict.update(All_Data_key)
    return(data_dict)

In [0]:
all_columns_euro = ['star_rating', 'Adult Occupant', 'Child Occupant', 'Vulnerable Road Users', 'Safety Assist', 'Tested Model', 'Body Type', 'Year Of Publication', 'Kerb Weight', 'VIN From Which Rating Applies', 'Class', 'Frontal Offset Deformable', 'Frontal Full Width', 'Lateral Impact', 'Whiplash Rear Impact', 'AEB City', 'Crash Test Performance', 'Safety Features', 'CRS Installation Check', 'Pedestrian', 'AEB Vulnerable Road', 'AEB Pedestrian', 'AEB Cyclist', 'Speed Assistance', 'Seat Belt Reminder', 'Lane Support', 'AEB Inter-Urban']
df_euro = pd.DataFrame(columns = all_columns_euro)

In [0]:
for link in euro_list_new:
    print(link)
    raw_euro_data = requests.get(link, headers = {'User-Agent': user_agent_ret()})
    data_dict = euro_ncap_scrapper(raw_euro_data.text)
    temp_euro_list = []
    for cols in all_columns_euro:
        try:
            temp_euro_list.append(data_dict[cols])
        except:
            temp_euro_list.append("Not_available")
    df_euro.loc[len(df_euro)] = temp_euro_list
    time.sleep(4)

https://www.euroncap.com/en/results/audi/a4/a5/25897
https://www.euroncap.com/en/results/citro\xc3\xabn/spacetourer/22856
https://www.euroncap.com/en/results/peugeot/traveller/22810
https://www.euroncap.com/en/results/toyota/proace/22857
https://www.euroncap.com/en/results/bmw/x1-/-x2/30140
https://www.euroncap.com/en/results/bmw/z4/22022
https://www.euroncap.com/en/results/infiniti/q30/22031
https://www.euroncap.com/en/results/jaguar/xe/22028
https://www.euroncap.com/en/results/jaguar/xf/22029
https://www.euroncap.com/en/results/kia/optima/22032
https://www.euroncap.com/en/results/kia/sportage/22033
https://www.euroncap.com/en/results/lancia/ypsilon/22030
https://www.euroncap.com/en/results/lexus/rx/22034
https://www.euroncap.com/en/results/mercedes-benz/glc/21987
https://www.euroncap.com/en/results/mini/clubman/22036
https://www.euroncap.com/en/results/nissan/navara/22037
https://www.euroncap.com/en/results/opel/vauxhall/astra/22038
https://www.euroncap.com/en/results/renault/megane/

In [0]:
df_euro.to_csv("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP_01.csv")
files.download("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP_01.csv")

MessageError: ignored

In [0]:
from google.colab import files
files.download("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP.csv")

In [0]:
all_link_text

In [0]:
# import requests
# link1 = "https://www.euroncap.com/en/results/aiways/u5/39672"
# raw_data = requests.get(link1)
# #soup_29 = BeautifulSoup(raw_data.text, "html.parser")
# euro_ncap_scrapper(raw_data.text)

In [0]:
# file_19_1 = open("/mntDrive/My Drive/Colab Notebooks/euro_19_2.html", "w")
# file_19_1.write(raw_data.text)
# file_19_1.close()
# files.download("/mntDrive/My Drive/Colab Notebooks/euro_19_2.html")

In [0]:
files.download("/mntDrive/My Drive/Colab Notebooks/Euro_NCAP_01.csv")